<a href="https://colab.research.google.com/github/OshanJayawardana/vip-cup/blob/main/FusingGlobalandLocal-main/Global_Local_Fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch torchvision -f https://download.pytorch.org/whl/torch_stable.html
!pip install opencv-python
!pip install scipy
!pip install imageio
!pip install tensorboardX
!pip install python-util

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
import os
import shutil
import random
random.seed(42)
import tensorboardX

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip "/content/gdrive/MyDrive/Datasets/pro_dataset.zip" -d "/content/pro_data/" &> /dev/null

In [ ]:
REAL = "/content/pro_data/data_pro/real/"
FAKE = "/content/pro_data/data_pro/synt/"

print(f"{len(os.listdir(REAL))} original real images")
print(f"{len(os.listdir(FAKE))} original fake images\n")

# 123322 original real images
# 115999 original fake images

123322 original real images
115999 original fake images



In [ ]:
root_dir = '/content/dataset'
os.makedirs(root_dir)

def create_train_val_dirs(root_path):
  path_train_real = os.path.join(root_path, "train/0_real")
  path_val_real = os.path.join(root_path, "val/0_real")
  path_train_fake = os.path.join(root_path, "train/1_fake")
  path_val_fake = os.path.join(root_path, "val/1_fake")

  os.makedirs(path_train_real)
  os.makedirs(path_val_real)
  os.makedirs(path_train_fake)
  os.makedirs(path_val_fake)

def split_data(SOURCE, TRAINING, VALIDATION, SPLIT_SIZE):

  files = []
  content_lst = os.listdir(SOURCE)

  for file in content_lst:
    file_path = os.path.join(SOURCE, file)
    if os.path.getsize(file_path) == 0:
      print("filename is zero length, so ignoring.")
    else:
      files.append(file)

  training_len = int(len(files) * SPLIT_SIZE)
  val_len = training_len + int(len(files) * ((1 - SPLIT_SIZE)/2))
  shuffled_lst = random.sample(files, len(files))

  train_set = shuffled_lst[ : training_len]
  val_set = shuffled_lst[training_len : val_len]

  for train_file in train_set: 
    train_path = os.path.join(SOURCE, train_file)
    dest_path = os.path.join(TRAINING, train_file)
    shutil.copyfile(train_path, dest_path)
    os.remove(train_path)

  for val_file in val_set: 
    val_path = os.path.join(SOURCE, val_file)
    dest_path = os.path.join(VALIDATION, val_file)
    shutil.copyfile(val_path, dest_path)
    os.remove(val_path)

In [ ]:
create_train_val_dirs(root_path=root_dir)

In [ ]:
REAL_SOURCE_DIR = "/content/pro_data/data_pro/real/"
FAKE_SOURCE_DIR = "/content/pro_data/data_pro/synt/"

TRAINING_DIR = "/content/dataset/train/"
VAL_DIR = "/content/dataset/val/"

TRAINING_REAL_DIR = os.path.join(TRAINING_DIR, "0_real/")
VAL_REAL_DIR = os.path.join(VAL_DIR, "0_real/")

TRAINING_FAKE_DIR = os.path.join(TRAINING_DIR, "1_fake/")
VAL_FAKE_DIR = os.path.join(VAL_DIR, "1_fake/")

split_size = .8

split_data(REAL_SOURCE_DIR, TRAINING_REAL_DIR, VAL_REAL_DIR, split_size)
split_data(FAKE_SOURCE_DIR, TRAINING_FAKE_DIR, VAL_FAKE_DIR, split_size)

print(f"{len(os.listdir(REAL_SOURCE_DIR))} remaining real images in original location ")
print(f"{len(os.listdir(FAKE_SOURCE_DIR))} remaining fake images in original location\n")

print(f"{len(os.listdir(TRAINING_REAL_DIR))} real images for training")
print(f"{len(os.listdir(TRAINING_FAKE_DIR))} fake images for training")
print(f"{len(os.listdir(VAL_REAL_DIR))} real images for validation")
print(f"{len(os.listdir(VAL_FAKE_DIR))} fake images for validation")

['img046282.jpg', 'img001646.jpg', 'img005932.jpg', 'img011243.jpg', 'img076624.jpg', 'img021984.jpg', 'img039940.jpg', 'img057234.jpg', 'img053862.jpg', 'img101162.jpg']
['img056812.jpg', 'img100127.jpg', 'img050284.jpg', 'img011366.jpg', 'img036421.jpg', 'img021985.jpg', 'img033047.jpg', 'img078423.jpg', 'img087964.jpg', 'img047714.jpg']
['img100272.jpg', 'img110758.jpg', 'img027952.jpg', 'img113151.jpg', 'img055311.jpg', 'img044279.jpg', 'img025792.jpg', 'img032109.jpg', 'img083317.jpg', 'img101079.jpg']
['img073483.jpg', 'img017832.jpg', 'img115953.jpg', 'img102243.jpg', 'img103132.jpg', 'img070791.jpg', 'img022416.jpg', 'img039184.jpg', 'img013514.jpg', 'img075592.jpg']
12333 remaining real images in original location 
11601 remaining fake images in original location

98657 real images for training
92799 fake images for training
12332 real images for validation
11599 fake images for validation


In [ ]:
os.rename("/content/pro_data","/content/test")
os.rename("/content/test/data_pro","/content/test/general")
os.rename("/content/test/arch1/real","/content/test/arch1/0_real")
os.rename("/content/test/arch1/synt","/content/test/arch1/1_fake")

In [ ]:
%cd "/content/gdrive/MyDrive/FusingGlobalandLocal"

/content/gdrive/MyDrive/FusingGlobalandLocal


## Train model

In [ ]:
!export CUDA_VISIBLE_DEVICES=0;
!python train.py --name test_2 --blur_prob 0.1 --blur_sig 0.0,3.0 --jpg_prob 0.1 --jpg_method cv2,pil --niter 5 --jpg_qual 30,100 --dataroot /content/dataset --batch_size 16 --lr 0.0001 --gpu_ids 0

----------------- Options ---------------
                     arch: res50                         
               batch_size: 16                            	[default: 64]
                    beta1: 0.9                           
                blur_prob: 0.1                           	[default: 0]
                 blur_sig: 0.0,3.0                       	[default: 0.5]
          checkpoints_dir: ./checkpoints                 
                class_bal: False                         
           continue_train: False                         
                 cropSize: 200                           
                 data_aug: False                         
                 dataroot: /content/dataset              	[default: ./dataset/]
          earlystop_epoch: 10                            
                    epoch: latest                        
              epoch_count: 1                             
                  gpu_ids: 0                             
                init_gai

## Testing

In [ ]:
!export CUDA_VISIBLE_DEVICES=0;

!python eval.py 

testing progan-generated images
directory, realimg, fakeimg: /content/test/progan 12333 11601
>>> Size of dataset:  23934
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/content/gdrive/MyDrive/FusingGlobalandLocal/networks/trainer.py:47: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, r